In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

2023-11-15 14:00:46.394031: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 14:00:46.431770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 14:00:46.432328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 14:00:47.044324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_PATH = 'imgs_zip/imgs/train/'
VALIDATION_PATH = 'imgs_zip/imgs/validation/'
TEST_PATH = 'imgs_zip/imgs/test/'

BATCH_SIZE = 32
IMG_SIZE = (324, 324)
NUM_CLASSES = len(next(os.walk(DATA_PATH))[1])

In [3]:
image_path = 'imgs_zip/imgs/train/'

train_dataset = []
for image_path in os.path.join(image_path, image_path):
    # Check if the image file exists
    if os.path.isfile(image_path):
        train_dataset.append(tf.keras.preprocessing.image.load_img(image_path))

train_labels = np.asarray([image['label'] for image in train_dataset])

np.save('train_labels.npy', train_labels)

train_labels = np.asarray([image['label'] for image in train_dataset])

In [4]:
# Load the pre-trained model
base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet')

# Freeze the pre-trained model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
global_average_pooling = tf.keras.layers.GlobalAveragePooling2D()

# Add a dense layer with 1024 units
dense_layer1 = tf.keras.layers.Dense(1024, activation='relu')

# Add a dropout layer with 0.2 dropout rate
dropout_layer1 = tf.keras.layers.Dropout(0.2)

# Add a dense layer with 512 units
dense_layer2 = tf.keras.layers.Dense(512, activation='relu')

# Add a dropout layer with 0.2 dropout rate
dropout_layer2 = tf.keras.layers.Dropout(0.2)

# Add a dense layer with NUM_CLASSES units
output_layer = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')

# Create the model
model = tf.keras.models.Sequential([
    base_model,
    global_average_pooling,
    dense_layer1,
    dropout_layer1,
    dense_layer2,
    dropout_layer2,
    output_layer
])

# Convert train_labels to a tensor
train_labels_tensor = tf.convert_to_tensor(train_labels)

# Get unique labels and their counts
unique, _, counts = tf.unique_with_counts(train_labels_tensor)

# Calculate class weights
class_weights = tf.math.reciprocal(tf.cast(counts, dtype=tf.float32))
class_weights = class_weights / tf.reduce_max(class_weights)

# Ensure class weights are in the right format for training
class_weights_dict = {i: weight.numpy() for i, weight in enumerate(class_weights)}

# weighted_categorical_crossentropy = tf.keras.losses.CategoricalCrossentropy(from_logits=True, class_weights=class_weights)

# Compile the model with the updated class weights
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create a weighted categorical crossentropy loss function
# class_weights = tf.cast(tf.math.reciprocal(tf.math.reduce_mean(np.unique(train_labels, return_counts=True)[1])), dtype=tf.float32)
# weighted_categorical_crossentropy = tf.keras.losses.CategoricalCrossentropy(from_logits=True, class_weights=class_weights)

# Create a class-balanced sampler
# class_weights = class_weights / np.max(class_weights)
# class_weights = np.power(class_weights, 1/3)
# class_weights = tf.cast(class_weights, dtype=tf.float32)
# class_balanced_sampler = tf.keras.utils.ClassWeight(class_weights)

# Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define a learning rate scheduler
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-5
)

# Create a data augmentation pipeline
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

# Train the model
history = model.fit(
    train_datagen.flow_from_directory(DATA_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'),
    epochs=20,
    class_weight=class_weights_dict,  # Apply class weights here
    validation_data=validation_datagen.flow_from_directory(VALIDATION_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'),
    callbacks=[reduce_lr]
)

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_loss, test_accuracy = model.evaluate(
    test_datagen.flow_from_directory(TEST_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
)

Found 2497 images belonging to 15 classes.
Found 206 images belonging to 15 classes.
Epoch 1/20


2023-11-15 14:01:45.680899: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 214990848 exceeds 10% of free system memory.
2023-11-15 14:01:45.735323: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 214990848 exceeds 10% of free system memory.
2023-11-15 14:01:45.813490: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 214990848 exceeds 10% of free system memory.
2023-11-15 14:01:45.849836: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 214990848 exceeds 10% of free system memory.
2023-11-15 14:01:45.881277: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 214990848 exceeds 10% of free system memory.
/home/sanarip03/.local/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


79/79 [==============================] - 628s 8s/step - loss: 2.6787 - accuracy: 0.1065 - val_loss: 2.7238 - val_accuracy: 0.0874 - lr: 0.0010
Epoch 2/20
79/79 [==============================] - 617s 8s/step - loss: 2.6422 - accuracy: 0.1173 - val_loss: 2.7367 - val_accuracy: 0.0874 - lr: 0.0010
Epoch 3/20
79/79 [==============================] - 620s 8s/step - loss: 2.6333 - accuracy: 0.1177 - val_loss: 2.7550 - val_accuracy: 0.0874 - lr: 0.0010
Epoch 4/20
79/79 [==============================] - 616s 8s/step - loss: 2.6267 - accuracy: 0.1286 - val_loss: 2.7486 - val_accuracy: 0.0874 - lr: 2.0000e-04
Epoch 5/20
79/79 [==============================] - 616s 8s/step - loss: 2.6215 - accuracy: 0.1185 - val_loss: 2.7520 - val_accuracy: 0.0874 - lr: 2.0000e-04
Epoch 6/20
79/79 [==============================] - 616s 8s/step - loss: 2.6184 - accuracy: 0.1229 - val_loss: 2.7463 - val_accuracy: 0.0874 - lr: 4.0000e-05
Epoch 7/20
19/79 [======>.......................] - ETA: 7:22 - loss: 2.616

KeyboardInterrupt: 

In [13]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [5]:
!python3 -m pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
filelock 3.12.3 requires typing-extensions>=4.7.1; python_version < "3.11", but you have typing-extensions 4.5.0 which is incompatible.


In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('CPU')))

Num GPUs Available:  1
